In [7]:
import pandas as pd

# Assuming you have the dataset in a CSV file
file_path = '3sn.csv'

# Load the dataset
df = pd.read_csv(file_path, encoding='latin1')
print("Successfully read the file with encoding 'latin1'.")

# Replace 'Category', 'Cost', and 'NetBookValue' with the actual column names from your dataset
category_column = 'CATEGORY'
cost_column = 'COST'
nbv_column = 'NBV'

# Count the number of assets per category
asset_count = df.groupby(category_column).size().reset_index(name='Asset Count')

# Calculate the total cost per category
total_cost = df.groupby(category_column)[cost_column].sum().reset_index(name='Total Cost')

# Calculate the total NBV per category
total_nbv = df.groupby(category_column)[nbv_column].sum().reset_index(name='Total NBV')

# Merge the results into a single DataFrame
result = asset_count.merge(total_cost, on=category_column).merge(total_nbv, on=category_column)

# Fill missing categories with 0 if necessary
all_categories = df[category_column].unique()
result = result.set_index(category_column).reindex(all_categories, fill_value=0).reset_index()
result.to_csv('3emptysn.CSV', index=False)
print(result)


C:\Users\kkibet\AppData\Local\Temp\ipykernel_11872\2452914223.py:7: DtypeWarning: Columns (6,7,8,10,15,16,17,18,19,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin1')


Successfully read the file with encoding 'latin1'.
                                              CATEGORY  Asset Count  \
0                                        1000-11090-10        29554   
1                                        1000-12001-20        40841   
2                                        1000-11099-30        81561   
3    leasehold improvemnt-Leasehold Improvmnt.-GENERAL           55   
4                                        1000-11002-20       161619   
..                                                 ...          ...   
114                                      1000-13099-30          101   
115                                      1000-12004-20          183   
116                                      3000-31000-20            4   
117                                      1000-13005-10            2   
118                                      4000-41000-20           31   

       Total Cost     Total NBV  
0    2.212343e+08  1.133399e+08  
1    1.493506e+07  5.505230e

In [23]:
import pandas as pd

file_path = '1.csv'

# Load the dataset
df2 = pd.read_csv(file_path, encoding='latin1')

# Step 2: Filter rows where SERIAL_NUMBER is missing (or empty)
df_missing_serial = df2[df2['SERIAL_NUMBER'].isnull() | (df2['SERIAL_NUMBER'] == '')]

# Step 3: Group by CATEGORY and count occurrences
category_counts = df_missing_serial.groupby('CATEGORY').size()

# Step 4: Reindex to include all categories, even those with count 0
all_categories = df2['CATEGORY'].unique()
category_counts = category_counts.reindex(all_categories, fill_value=0)

# Step 5: Convert the Series to a DataFrame (optional)
category_counts_df = category_counts.reset_index(name='1count_missing_serial_numbers')

category_counts_df.to_csv('1c.CSV', index=False)

print(category_counts_df)


C:\Users\kkibet\AppData\Local\Temp\ipykernel_22132\1415732256.py:6: DtypeWarning: Columns (6,7,8,10,15,16,17,18,19,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path, encoding='latin1')


                CATEGORY  1count_missing_serial_numbers
0          1000-12001-20                          33863
1         1000-11002.-10                          14916
2          1000-12090-10                          68267
3          1000-12001-10                          15515
4    IT-Printers-GENERAL                              4
..                   ...                            ...
171        1000-18100-30                              2
172       6000-62000.-10                              0
173        4000-48300-30                              1
174        5000-51000-50                              0
175        1000-14004-10                              0

[176 rows x 2 columns]


In [13]:
import pandas as pd

# Load the CSV files into pandas DataFrames
df1 = pd.read_csv('3emptysn.csv', encoding='latin1')
df2 = pd.read_csv('2emptysn.csv', encoding='latin1')
df3 = pd.read_csv('1emptysn.csv', encoding='latin1')

# Merge DataFrames on 'CATEGORY' column using outer join to include all categories
merged_df = pd.merge(df1, df2, on='CATEGORY', how='outer', suffixes=('_file1', '_file2'))
merged_df = pd.merge(merged_df, df3, on='CATEGORY', how='outer')

# Fill NaN values with 0 for numerical columns
for col in ['Asset Count', 'Total Cost', 'Total NBV']:
    if col in merged_df.columns:
        merged_df[col + '_file1'].fillna(0, inplace=True)
        merged_df[col + '_file2'].fillna(0, inplace=True)
        merged_df[col + '_file3'].fillna(0, inplace=True)
        merged_df[col] = merged_df[[col + '_file1', col + '_file2', col + '_file3']].sum(axis=1)

# Drop the individual file columns
merged_df.drop(columns=[col + '_file1' for col in ['Asset Count', 'Total Cost', 'Total NBV']] +
                              [col + '_file2' for col in ['Asset Count', 'Total Cost', 'Total NBV']] +
                              [col + '_file3' for col in ['Asset Count', 'Total Cost', 'Total NBV']], inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)



C:\Users\kkibet\AppData\Local\Temp\ipykernel_11872\3093565890.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[col + '_file1'].fillna(0, inplace=True)
C:\Users\kkibet\AppData\Local\Temp\ipykernel_11872\3093565890.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

KeyError: 'Asset Count_file3'